In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Import the file
Zipcode_df = pd.read_csv("Final_SE_df.csv", converters={'Zipcode': lambda x: str(x)})
Zipcode_df.head()

,Zipcode,State,Urban Type,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,ZC Eth,Pct Dom Eth,...,Lng_Pct,House_Const_Yr,Med Gross Rent,Med Home_Val,Household Income,Per Capita Income,Pvty Rate,Unemployed Rate,Pov_Group,Pov_Pct
0,00601,PR,Mixed,18.180555,-66.749961,17982.0,279.449245,37.6,His,99.777555,...,87.932377,1982.0,359.0,104500.0,10816.0,7453.0,60.399288,12.306751,His,99.631710
1,00602,PR,Urbanized Area,18.361945,-67.175597,40260.0,1315.127560,39.5,His,93.594138,...,91.308992,1980.0,395.0,90200.0,16079.0,8474.0,53.949329,9.456036,His,93.687845
2,00603,PR,Urbanized Area,18.455183,-67.119887,52408.0,1657.641700,39.9,His,96.239124,...,89.867959,1977.0,384.0,128400.0,16804.0,10179.0,48.578461,7.615250,His,97.022664
3,00606,PR,Mixed,18.158345,-66.932911,6331.0,149.637193,40.8,His,99.873638,...,91.154636,1978.0,280.0,108500.0,12512.0,5863.0,64.286843,2.827357,His,99.803440
4,00610,PR,Urbanized Area,18.295366,-67.125135,28328.0,788.729257,40.2,His,98.838605,...,92.452697,1979.0,424.0,113200.0,17475.0,8452.0,49.438718,5.277464,His,98.821849


In [ ]:
# Let's look at some correlations

# Population vs Pop Density
PvPD = Zipcode_df["Population"].corr(Zipcode_df["Pop Density"]) # 0.361
MGRvMHV = Zipcode_df["Med Gross Rent"].corr(Zipcode_df["Med Home_Val"]) # 0.757
HIvPCI = Zipcode_df["Household Income"].corr(Zipcode_df["Per Capita Income"]) # 0.804
PCIvMGR = Zipcode_df["Per Capita Income"].corr(Zipcode_df["Med Gross Rent"]) # 0.645
PCIvMGR

In [12]:
Zipcode_df.count()

Zipcode              33120
State                33120
Urban Type           33120
Center Point Lat     33120
Center Point Lng     33120
Population           33120
Pop Density          33120
Median Age           32615
ZC Eth               33120
Pct Dom Eth          32810
Edu Score            33120
Main_Lang            33120
Lng_Pct              32810
House_Const_Yr       32230
Med Gross Rent       27431
Med Home_Val         30793
Household Income     31067
Per Capita Income    32542
Pvty Rate            32810
Unemployed Rate      32810
Pov_Group            33120
Pov_Pct              33120
dtype: int64

In [19]:
Zipcode_test = Zipcode_df.loc[Zipcode_df["Population"] != 0]
# Create a correlation matrix to see if any of our variables show the same thing
corr = Zipcode_test.corr()
corr.style.background_gradient()
# Household income and per capita income are highly correlated (0.804), others that are not great are Per Cap Inc and Med Gross Rent
# Med Home Val and Med Gross Rent

,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,Pct Dom Eth,Edu Score,Lng_Pct,House_Const_Yr,Med Gross Rent,Med Home_Val,Household Income,Per Capita Income,Pvty Rate,Unemployed Rate,Pov_Pct
Center Point Lat,1,-0.104016,-0.202083,-0.027416,0.0916532,0.2628,0.0619282,0.179179,-0.28089,-0.0395281,0.0336411,0.139234,0.126342,-0.195975,-0.052685,0.126704
Center Point Lng,-0.104016,1,-0.0400006,0.0500838,0.0701242,0.06558,0.0061937,0.237991,-0.122537,-0.0350726,-0.103066,0.0437819,0.0423627,-0.0579994,-0.00318239,0.0279581
Population,-0.202083,-0.0400006,1,0.365579,-0.304662,-0.39167,0.270572,-0.467323,0.118937,0.363432,0.252029,0.144674,0.102955,0.0317936,0.0882962,-0.187552
Pop Density,-0.027416,0.0500838,0.365579,1,-0.183584,-0.280089,0.164755,-0.323021,-0.161574,0.294932,0.31686,0.0855637,0.132782,0.0379719,0.0696849,-0.166592
Median Age,0.0916532,0.0701242,-0.304662,-0.183584,1,0.297992,0.00878167,0.392591,-0.0665375,-0.0724941,0.0574571,0.0317586,0.25822,-0.212965,-0.141045,0.133155
Pct Dom Eth,0.2628,0.06558,-0.39167,-0.280089,0.297992,1,-0.077478,0.432035,-0.0772704,-0.212248,-0.118889,0.0633332,0.0841433,-0.210525,-0.168808,0.400296
Edu Score,0.0619282,0.0061937,0.270572,0.164755,0.00878167,-0.077478,1,-0.108792,0.0989841,0.62577,0.65471,0.647144,0.638996,-0.329429,-0.117419,-0.106068
Lng_Pct,0.179179,0.237991,-0.467323,-0.323021,0.392591,0.432035,-0.108792,1,-0.0368933,-0.337837,-0.249929,-0.0753061,0.00562594,-0.147037,-0.090523,0.15882
House_Const_Yr,-0.28089,-0.122537,0.118937,-0.161574,-0.0665375,-0.0772704,0.0989841,-0.0368933,1,0.163373,0.0649697,0.120963,0.0176877,-0.034666,0.0035265,-0.0238552
Med Gross Rent,-0.0395281,-0.0350726,0.363432,0.294932,-0.0724941,-0.212248,0.62577,-0.337837,0.163373,1,0.757784,0.723649,0.64524,-0.357166,-0.0421413,-0.248893


In [28]:
# Let's see if the Hispanic variable and the Language variable show the same thing
Testing_Lang_Eth = Zipcode_test.loc[Zipcode_test["ZC Eth"]=="His",:]
Testing_Lang_Eth["Main_Lang"].value_counts()
# No, it doesn't, of the zipcodes dominated by Hispanics, you've got about 50% Spanish dominant, 50% mixed

# Where are all the states where "Other" in languages is most dominant?
# Looks like AZ, AK, NM, so I bet this is where dual languages speakers (Eng/Sp) are the #1
Testing_Lang_2 = Zipcode_test.loc[Zipcode_test["Main_Lang"]=="Oth",:]
Testing_Lang_2["State"].value_counts()

#Let's try to bin a few indicators to see where we stand... Edu Scores, *Pct Dom Eth, and Pov Rate,
#Then I'd like to look at dominant ethnicity vs dominant poverty group
Zipcode_test["Pct Dom Eth"].min()
bins = [0,25,50,75,100]
bin_names = ["0-25", "25-50", "50-75", "75-100"]
Binned = pd.cut(Zipcode_test["Pct Dom Eth"], bins, labels=bin_names)
Zipcode_test["Eth_Bins"] = Binned

Eth_groups = Zipcode_test.groupby("Eth_Bins")

# Find how many rows fall into each bin
Ethnic_break = pd.DataFrame(Eth_groups["Zipcode"].count())
Ethnic_break["Zipcode"].sum() # 32807
Ethnic_break["Pct"] = Ethnic_break["Zipcode"] / 32807 * 100
Ethnic_break # So 81% of American zipcodes have 75% or more of one race/ethnicity, 13% have between 50 and 75% whites, 
             # 4% have between 25 and 50% whites

C:\Users\Brian Data Sci\Anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Zipcode,Pct
Eth_Bins,,
0-25,226,0.688877
25-50,1629,4.965404
50-75,4266,13.003322
75-100,26686,81.342396


In [29]:
# Let's use a groupby to look at zipcodes dominated by blacks or hispanics
Zipcode_test.groupby("ZC Eth").mean()

,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,Pct Dom Eth,Edu Score,Lng_Pct,House_Const_Yr,Med Gross Rent,Med Home_Val,Household Income,Per Capita Income,Pvty Rate,Unemployed Rate,Pov_Pct
ZC Eth,,,,,,,,,,,,,,,,
Black,35.503888,-85.585015,16113.621622,3652.939117,36.668793,86.460715,46.779270,88.951394,1967.649533,762.397260,99361.691542,31784.891358,16694.112903,31.046235,8.058506,85.168766
His,29.275521,-96.544206,23074.651724,3178.924701,34.391916,90.363357,44.206442,58.675523,1975.538188,709.467641,130545.537002,32150.326415,14229.467257,32.807868,5.193471,90.159906
Mixed,36.376453,-92.108764,18731.836631,3644.597485,37.499738,56.212273,52.567869,74.315536,1973.735566,937.547972,200517.051557,50103.970805,24553.314970,19.154760,4.679386,61.497805
Other,45.020051,-129.823454,5654.098592,855.712133,31.429944,91.100867,44.327875,63.177328,1981.054131,693.980263,149906.489164,39559.529586,15825.398860,31.780304,7.993738,92.222457
White,39.626497,-90.175992,7228.392045,670.315724,43.385018,93.127449,51.919025,88.135551,1971.316907,806.745701,171400.419230,55274.376538,28044.076515,12.920184,3.360793,84.986090


In [43]:
# Bin Edu Scores: Most zipcodes are really around the middle on scores
Zipcode_test_2 = Zipcode_test.copy()
bins = [0,25,50,75,100]
bin_names = ["0-25", "25-50", "50-75", "75-100"]
Binned = pd.cut(Zipcode_test_2["Edu Score"], bins, labels=bin_names)
Zipcode_test_2["Edu_Bins"] = Binned

Edu_groups = Zipcode_test_2.groupby("Edu_Bins")

# Find how many rows fall into each bin
Edu_break = pd.DataFrame(Edu_groups["Zipcode"].count())
Edu_break["Zipcode"].sum() #32692
Edu_break["Pct"] = Edu_break["Zipcode"] / 32692 * 100
Edu_break

,Zipcode,Pct
Edu_Bins,,
0-25,27,0.082589
25-50,15989,48.907990
50-75,15932,48.733635
75-100,744,2.275786


In [48]:
# Bin Poverty Scores: The vast majority of neighborhoods have less than 25% poverty rate
Zipcode_test_3 = Zipcode_test.copy()
bins = [0,25,50,75,100]
bin_names = ["0-25", "25-50", "50-75", "75-100"]
Binned = pd.cut(Zipcode_test_3["Pvty Rate"], bins, labels=bin_names)
Zipcode_test_3["Pov_Bins"] = Binned

Pov_groups = Zipcode_test_3.groupby("Pov_Bins")

# Find how many rows fall into each bin
Pov_break = pd.DataFrame(Pov_groups["Zipcode"].count())
Pov_break["Zipcode"].sum() #31328
Pov_break["Pct"] = Pov_break["Zipcode"] / 31328 * 100
Pov_break

,Zipcode,Pct
Pov_Bins,,
0-25,26554,84.761236
25-50,4265,13.614019
50-75,411,1.311925
75-100,98,0.312819


In [58]:
# Ethnic poverty
Hispanic_set = Zipcode_test.loc[Zipcode_test["ZC Eth"]=="His",:]
Hispanic_dom_pov = pd.DataFrame(Hispanic_set["Pov_Group"].value_counts())
his_len = Hispanic_dom_pov["Pov_Group"].sum()
Hispanic_dom_pov["Pct"] = Hispanic_dom_pov["Pov_Group"] / his_len
Hispanic_dom_pov #82% of Hispanic dominated neighborhoods have mostly Hispanic poverty

Black_set = Zipcode_test.loc[Zipcode_test["ZC Eth"]=="Black",:]
Black_dom_pov = pd.DataFrame(Black_set["Pov_Group"].value_counts())
black_len = Black_dom_pov["Pov_Group"].sum()
Black_dom_pov["Pct"] = Black_dom_pov["Pov_Group"] / black_len
Black_dom_pov #94% of Black dominated neighborhoods have mostly Black poverty

White_set = Zipcode_test.loc[Zipcode_test["ZC Eth"]=="White",:]
White_dom_pov = pd.DataFrame(White_set["Pov_Group"].value_counts())
white_len = White_dom_pov["Pov_Group"].sum()
White_dom_pov["Pct"] = White_dom_pov["Pov_Group"] / white_len
White_dom_pov #92% of White dominated neighborhoods have mostly White poverty

Mixed_set = Zipcode_test.loc[Zipcode_test["ZC Eth"]=="Mixed",:]
Mixed_dom_pov = pd.DataFrame(Mixed_set["Pov_Group"].value_counts())
mixed_len = Mixed_dom_pov["Pov_Group"].sum()
Mixed_dom_pov["Pct"] = Mixed_dom_pov["Pov_Group"] / mixed_len
Mixed_dom_pov #44% of Mixed neighborhoods have mostly White poverty, 31% have mostly Black poverty, 10% have mostly Hispanic poverty

,Pov_Group,Pct
White,2746,0.449060
Black,1915,0.313164
His,644,0.105315
Other,571,0.093377
No_Pvty,222,0.036304
Mixed B-W,14,0.002289
Mixed W-O,3,0.000491
